In [1]:
sentence = ["Amidst the tranquil beauty of a sun-dappled forest, where the gentle rustle of leaves harmonizes with the chorus of birdsong, a solitary figure wanders along a winding path. The earth beneath their feet is soft and mossy, carpeted with a tapestry of ferns and wildflowers. Shafts of golden light filter through the canopy above, casting intricate patterns on the forest floor. In this sanctuary of nature, time seems to slow, and the worries of the world fade into insignificance. Here, amidst the whispering trees and the babbling brooks, there is a sense of peace—a timeless tranquility that soothes the."]

import time

UAE-Large-V1 #2


In [2]:
from angle_emb import AnglE

angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

/Users/jiaweihong/Desktop/mmqg/.venv/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [3]:
start = time.time()
vec = angle.encode(sentence, to_numpy=True)
end = time.time()
print("Time taken: ", end - start)

Time taken:  1.5670180320739746


SFR-Embedding-Mistral #1

In [2]:

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("Salesforce/SFR-Embedding-Mistral")



README.md:   0%|          | 0.00/85.3k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

: 

In [1]:
start = time.time()
  
# Embedding
embeddings = model.encode("sentence")

end = time.time()
print("Time taken: ", end - start)

NameError: name 'time' is not defined


bge-large-en-v1.5 #3


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-zh-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-large-zh-v1.5')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [ ]:
start = time.time()
# Tokenize sentences
encoded_input = tokenizer(sentence, padding=True, truncation=True, max_length=512, return_tensors='pt')
# for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
end = time.time()
print("Time taken: ", end - start)

Time taken:  5.155472755432129


GIST-large-Embedding-v0 #4

In [ ]:
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

revision = None  # Replace with the specific revision to ensure reproducibility in  case the model is updated.

model = SentenceTransformer("avsolatorio/GIST-large-Embedding-v0", revision=revision)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: avsolatorio/GIST-large-Embedding-v0


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps


In [ ]:
start = time.time()

# Compute embeddings
embeddings = model.encode(sentence, convert_to_tensor=True)

end = time.time()
print("Time taken: ", end - start)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Time taken:  0.4793388843536377


bge-base-en-v1.5 #5

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-base-zh-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-base-zh-v1.5')
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
start = time.time()
# Tokenize sentences
encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
# for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
end = time.time()
print("Time taken: ", end - start)

Time taken:  1.337890863418579


mxbai-embed-large-v1 #2

In [ ]:
from sentence_transformers import SentenceTransformer
# 1. load model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-2d-large-v1")

# 2. Encode
# start = time.time()

# embeddings = model.encode(sentence)
# end = time.time()
# print("Time taken: ", end - start)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

You try to use a model that was created with version 2.5.1, however, your version is 2.3.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/107k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

TypeError: Pooling.__init__() got an unexpected keyword argument 'include_prompt'